# Objetivo do Notebook

Neste arquivo, o objetivo é documentar e testar a API ('app/api_modelo.py') configurada para o modelo treinado no jupyter notebook 'notebooks/estudo_oms.ipynb'.

# 0.Imports

Importamos a lib requests para enviar o JSON de teste do modelo

In [7]:
import requests

# 1. Configurando a API

### Documentação da API

- #### Execução da API

Arquivo: app/api_modelo.py

- #### Endpoint

> POST /predict

- #### Descrição
Esse endpoint recebe uma requisição com um vetor de características (features) sobre duas pessoas (V1 e V2) e retorna a predição da probabilidade de 'taxa de contagio' entre elas, segundo o modelo treinado.

- #### Formato do JSON de entrada
O JSON enviado deve conter um único campo chamado "data", cujo valor é uma lista ordenada de 20 elementos, representando as seguintes variáveis:

| Ordem | Nome                            | Tipo    | Descrição                                            |
| ----- | ------------------------------- | ------- | ---------------------------------------------------- |
| 1     | grau                            | `str`   | Grau de relacionamento (ex: `"trabalho"`)            |
| 2     | proximidade                     | `str`   | Frequência de contato (ex: `"visita_frequente"`)     |
| 3     | idade\_V1                       | `int`   | Idade da pessoa V1                                   |
| 4     | estado\_civil\_V1               | `str`   | Estado civil de V1 (ex: `"solteiro"`, `"casado"`)    |
| 5     | qt\_filhos\_V1                  | `int`   | Quantidade de filhos de V1                           |
| 6     | estuda\_V1                      | `int`   | Estuda? 1 = Sim, 0 = Não                             |
| 7     | trabalha\_V1                    | `int`   | Trabalha? 1 = Sim, 0 = Não                           |
| 8     | pratica\_esportes\_V1           | `int`   | Pratica esportes? 1 = Sim, 0 = Não                   |
| 9     | transporte\_mais\_utilizado\_V1 | `str`   | Meio de transporte usado por V1 (ex: `"particular"`) |
| 10    | IMC\_V1                         | `float` | Índice de Massa Corporal (IMC) de V1                 |
| 11    | idade\_V2                       | `int`   | Idade da pessoa V2                                   |
| 12    | estado\_civil\_V2               | `str`   | Estado civil de V2                                   |
| 13    | qt\_filhos\_V2                  | `int`   | Quantidade de filhos de V2                           |
| 14    | estuda\_V2                      | `int`   | Estuda? 1 = Sim, 0 = Não                             |
| 15    | trabalha\_V2                    | `int`   | Trabalha? 1 = Sim, 0 = Não                           |
| 16    | pratica\_esportes\_V2           | `int`   | Pratica esportes? 1 = Sim, 0 = Não                   |
| 17    | transporte\_mais\_utilizado\_V2 | `str`   | Meio de transporte usado por V2                      |
| 18    | IMC\_V2                         | `float` | Índice de Massa Corporal (IMC) de V2                 |


- #### Exemplo de JSON válido:

{
  "data": ["familia", "visita_rara", 32, "solteiro", 0, 1, 1, 1, "particular", 24.5,
           29, "casado", 1, 0, 1, 0, "taxi", 22.3]
}

- #### Resposta (JSON)

{
  "prediction": 0.0573
}
prediction: Valor predito pelo modelo (probabilidade estimada de relacionamento entre V1 e V2).

- #### Possíveis erros

| Código | Motivo                                        |
| ------ | --------------------------------------------- |
| 400    | JSON malformado ou quantidade errada de dados |
| 500    | Erro interno do modelo ou do processamento    |

# 2. Executando a API

Para iniciar a API deste projeto, basta executar o script 'app/api_modelo.py'. Esse arquivo utiliza Flask para disponibilizar a API, permitindo a realização de testes com o modelo treinado.

In [8]:
# Declarando a url de chamado da API

url = 'http://127.0.0.1:8000/predict'

# 3. Testes

In [ ]:
# Teste com JSON em formato correto

payload = {
    "data": ["trabalho", "visita_frequente", 40, "casado", 2, 0, 1, 0, "particular", 27.8,
             35, "solteiro", 0, 1, 1, 1, "particular", 23.1]
}
response = requests.post(url, json=payload)
print("Teste 1:", response.json())

Teste 1: {'prediction': 0.3793753981590271}


In [15]:
# Teste com JSON em formato correto

payload = {
    "data": ["trabalho", "visita_frequente", 99, "divorciado", 8, 0, 0, 1, "particular", 18.0,
             5, "solteiro", 0, 1, 0, 1, "particular", 30.0]
}
response = requests.post(url, json=payload)
print("Teste 2:", response.json())

Teste 2: {'prediction': 0.8849464654922485}


In [14]:
# Teste de JSON inválido (campo 'data' ausente)

payload = {
    "input": ["trabalho", "visita_frequente", 32, "solteiro", 0, 1, 1, 1, "particular", 24.5,
              29, "casado", 1, 0, 1, 0, "particular", 22.3]
}
response = requests.post(url, json=payload)
print("Teste 3:", response.json())

Teste 3: {'error': "JSON deve conter o campo 'data'"}


In [13]:
# Teste de JSON inválido (quantidade incorreta de features, 17 ao invés de 18)

payload = {
    "data": ["amigos", "visita_rara", 32, "solteiro", 0, 1, 1, 1, "publico", 24.5,
             29, "casado", 1, 0, 1, 0, "publico"]  # Faltando IMC_V2
}
response = requests.post(url, json=payload)
print("Teste 4:", response.json())

Teste 4: {'error': 'Esperado 18 features, recebido 17'}


In [12]:
# Teste de JSON inválido (idade como string)

payload = {
    "data": ["amigos", "visita_rara", "trinta e dois", "solteiro", 0, 1, 1, 1, "publico", 24.5,
             29, "casado", 1, 0, 1, 0, "publico", 22.3]
}
response = requests.post(url, json=payload)
print("Teste 5:", response.json())

Teste 5: {'error': "Cannot use mean strategy with non-numeric data:\ncould not convert string to float: 'trinta e dois'"}


In [11]:
# Teste com JSON em formato correto

payload = {
    "data": ["amigos", "visita_rara", None, "casado", 2, 1, 1, 0, "publico", None,
             35, "solteiro", 0, 0, 1, 1, "publico", 25.0]
}
response = requests.post(url, json=payload)
print("Teste 6:", response.json())

Teste 6: {'prediction': 0.35981830954551697}
